In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, TensorDataset

from typing import List, Tuple, NoReturn, Any, Optional, Union
import pandas as pd
import numpy as np
import random

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from tqdm.auto import tqdm, trange
import os

import datasets
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, HfArgumentParser

In [3]:
dataset = load_from_disk("/opt/ml/data/train_dataset")
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [4]:
model_checkpoint = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [20]:
df = pd.read_csv("/opt/ml/data/bm25/bm25_top10_pp2_val.csv")
df.head()

,question,id,context_id,context,original_context,answers
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,[ 4459 4903 41809 13812 32041 42379 21471 50...,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'answer_start': [284], 'text': ['한보철강']}"
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,[47550 17714 19171 19173 19403 19174 33524 194...,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'answer_start': [146], 'text': ['1871년']}"
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,[11493 40651 20002 47787 5184 53943 285 45...,"촐라 정부 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 권...","촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'answer_start': [517], 'text': ['나뭇잎']}"
3,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,[55260 55251 55257 55261 55262 11850 55258 552...,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'answer_start': [1109], 'text': ['금대야']}"
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,[28787 20107 42013 55610 45043 5960 5743 423...,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'answer_start': [386], 'text': ['수평적 관계']}"


In [21]:
answers = df.answers.to_list()
question = df.question.to_list()
context_id = df.context_id.to_list()
original_context = df.original_context.to_list()
top_k_passage = []
target = []

In [22]:
len(answers), len(question), len(original_context)

(240, 240, 240)

In [23]:
for idx, passages in enumerate(df.context.tolist()):
    p_list = passages.split('▦')
    
    if original_context[idx] in p_list:
        top_k_passage.append(p_list)
        target_index = p_list.index(original_context[idx])
        target.append(target_index)
    else :
        p_list = [original_context[idx]] + p_list[:-1]
        top_k_passage.append(p_list)
        target.append(0)

In [26]:
top_k_passage[0]

['순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다. 경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인

In [25]:
i = random.randrange(240)
i = 0
print(question[i], answers[i], context_id[i])
print(original_context[i], '\n', '*'*150)
print(target[i], len(top_k_passage[i]))
print(top_k_passage[i][target[i]], '\n', '*'*150)
print(top_k_passage[i])


처음으로 부실 경영인에 대한 보상 선고를 받은 회사는? {'answer_start': [284], 'text': ['한보철강']} [ 4459  4903 41809 13812 32041 42379 21471  5079 40186 52223]
순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\n\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \n\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다. 
 ******************************************************************************************************************************************************
0 10
순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 

In [17]:
p_seqs = tokenizer(
            top_k_passage[i],
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_token_type_ids=False,
        )  # (top_k, 512)

q_seqs = tokenizer(
    question[i],
    padding="max_length",
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False ,
)  # (1, 512)

In [80]:
p_seqs.input_ids.shape

torch.Size([10, 512])

In [81]:
q_seqs.input_ids.shape

torch.Size([1, 512])

In [82]:
tokenizer.decode(p_seqs.input_ids[9])

'[CLS] 모부투는 위원들의 지원이 주로 젊은 대학 졸업생들이었던 위원 칼리지로 결성하면서 제1공화국 동안 잠시 다스렸다. 군인이자 정치적 인물 둘다인 모부투는 군사 독재자가 되는 데 관심이 없었다. 육군은 그의 주요 근거였으나 그는 자신의 복잡한 정치적 망상 조직을 지속하였고 정책을 만드는 정부에서 주요 참가자였다. 그는 1961년 8월 새로운 의회가 소집될 때까지 다스렸으며, 시릴 아둘라가 총리로서와 함께 새로운 정부가 형성되었다. 카사부부는 대통령으로서 남아있었다. 이 시기 동안 권력을 위하여 4개의 단체들 - 카사부부 아래 민간 정부, 앙투안 [UNK] 아래 북부 지방들, 카탕가에서 촘베와 알베르 칼론지에 의하여 지도된 카사이 주에서 분리 단체가 우열을 다투고 있었다. 촘베를 제외한 전부는 소란스러운 2년 동안 다스린 아둘라 아래 새로운 정부를 형성하는 데 가입하였다. 1964년 7월 촘베는 총리직을 취하고, " 국가 일치의 정부 " 로 알려진 새로운 국민 정부를 형성하는 데 촘베를 초청하였다. 8월 1일의 새 헌법은 연방주의의 조직을 합병한 대통령 제도를 설립하였다. = = 쿠데타를 이끌며 = = 제1공화국이었던 5년 완패 후, 모부투는 권력을 취하여 " 정치적 지도자들은 이 국가의 시민들의 평안을 위하여 아무 여김없이 투쟁하는 무효의 권력을 위하여 정착하였다. " 고 선언하였다. 1965년 쿠데타에 이어 모부투는 정부 그대로 제도적인 체제를 간직하였고, 새로운 공무원들과 함께 직위들을 채웠다. 그는 국가 원수가 되었고, 그의 부하 대령 ( 후에 장군 ) 레오나르 물람바는 총리로 임명되었다. 헌법적 정면의 뒤로 군인 직원은 유일한 효과적인 권한이 되었다. 모부투의 대통령직의 첫 5년은 그의 지배들과 직무로 들어가는 권력의 합동을 보았다. 가상 무정부 상태의 6년 후, 모부투는 자이르에 어떤 법률과 질서를 가져오는 데 성공하였다. 그가 국가 원수가 되었을 때 그는 " 5년 동안 국가 [SEP]'

In [11]:
file_name = "dksahjklvjxzckl \
asdkljklzxjckljvcxzk \
ksdajklxzcvjklvxcz\
fjklasdjkldscv\
dfsjkljxckzl"

In [12]:
file_name

'dksahjklvjxzckl asdkljklzxjckljvcxzk ksdajklxzcvjklvxczfjklasdjkldscvdfsjkljxckzl'